In [28]:
import numpy as np
import pandas as pd
import itertools
import time
from IPython.display import SVG
from scipy import sparse
import numpy as np
import networkx as nx
from math import inf, isinf

In [29]:
ordered_nodes = {v:[] for v in range(0,5)}
ordered_nodes[0].append(2)
ordered_nodes

{0: [2], 1: [], 2: [], 3: [], 4: []}

In [30]:
def load_graph(filename):
    """
    load a graph from file of format:
    src dest
    :param filename:
    :return: G: nx.graph, list of edges, list of nodes
    """
    #time_start = time.time()
    data_edge = pd.read_table(filename, skiprows=4,sep='\t', header=None)
    edges = data_edge.values.tolist()
    #print("Number of edges:", len(edges))
    nodes = set()
    for edge in edges:
        nodes.add(edge[0])
        nodes.add(edge[1])
    #print("Number of nodes:", len(nodes))
    # make id-index correspondence
    v = 0
    # dictionary id node:index
    node_to_index = {}
    for node in nodes:
        node_to_index[node] = v
        v += 1
    for edge in edges:
        edge[0] = node_to_index[edge[0]]
        edge[1] = node_to_index[edge[1]]
    nodes = list(node_to_index.values())
    nodes.sort()
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    #time_end = time.time()
    #print("Charge time:", time_end - time_start, "seconds")
    
    return G, edges, nodes
    

In [31]:
G,edges,nodes = load_graph('test.txt')
nodes = list(G.nodes)
edges = list(G.edges)
#np.zeros(len(nodes),int)
#G.remove_node(0)
#d_out=list(nx.degree(G))
d_out = {v: list(nx.all_neighbors(G, v)) for v in G}
len(d_out)
#min(d_out,key=lambda x:len(d_out[x]))

9

In [32]:
def remove_key(d, key):
    r = dict(d)
    del r[key]
    for name in r:
        value = r[name]
        if key in value:
            value.remove(key)
    return r

In [33]:
def Core_decomposition(G):
    graph = G.copy()
    nodes = list(graph.nodes)
    n = len(nodes)
    ord = list(np.zeros(len(nodes),int)) 
    corelist = list(np.zeros(len(nodes),int))
    c = 0
    d_out = {v: list(nx.all_neighbors(G, v)) for v in G}
    times = 0
    
    while len(d_out) != 0:
        print(times)
        times = times + 1
        minnode = min(d_out,key=lambda x:len(d_out[x]))
        #print(minnode)
        c = max(len(d_out[minnode]),c)
        corelist[minnode] = c
        ord[minnode] = n
        d_out= remove_key(d_out,minnode)
        n = n - 1
    
    return corelist,ord
        
        

In [35]:
def order_nodes_degree(graph):
    # number of nodes
    n = len(graph.nodes)
    # degrees[i] == (i, degree_node_i)
    degrees = list(nx.degree(graph))
    # (i, degree_node_i)
    max_degree = tuple(map(max, zip(*degrees)))[1]
    # sort nodes, by degree of each node
    sorted_node = [node for (node, degree) in sorted(degrees, key=lambda t:t[1])]

    # indexes where each degree section starts
    start_degrees_index = list(range(max_degree + 1))
    # location of each node in sorted_node
    node_index = list(range(n))

    # a 2-dimension array (dictionary in python) to represent the nodes having a same degree
    ordered_nodes = {i:[] for i in range(max_degree + 1)}

    for node in range(n):
        d = degrees[node][1]
        ordered_nodes[d].append(node)
    index = 0
    for d in range(max_degree + 1):
        start_degrees_index[d] = index
        for node in ordered_nodes[d]:
            node_index[node] = index
            index += 1
    return degrees, sorted_node, start_degrees_index, node_index

In [40]:
def Core_decompositionfinal(graph):
    """
    core decomposition of a graph
    :param graph: a networkx.Graph
    :return: a core value of graph
    """
    # core value
    c = 0
    # nodes
    nodes = list(graph.nodes)
    n = len(nodes)
    # core of each node, initialised to 0
    nodes_core = [(i, 0) for i in nodes]
    degrees, sorted_nodes, start_degrees_index, node_index = order_nodes_degree(graph)
    # reverse order of visiting each node
    ord = list(range(n))
    for i in range(n):
        u = sorted_nodes[i]
        u_degree = degrees[u][1]
        ord[u] = n-i
        visited_neighbour = False
        if u_degree > c:
            c = u_degree
        nodes_core[u] = (u, c)
        # remove node u from graph: set degree of node u to inf so not to be considered anymore
        degrees[u] = (u, float(inf))
        # iterate its neighbours to decrease their degree
        neighbours = list(graph.neighbors(u))
        for v in neighbours:
            v_degree = degrees[v][1]
            # consider only nodes with degree != inf
            if not isinf(v_degree):
                # as u is deleted, v's degree will decrease by 1
                degrees[v] = (v, v_degree - 1)
                if i < start_degrees_index[v_degree]:
                    a = start_degrees_index[v_degree]
                    b = node_index[v]
                    # swap the head of section with a
                    sorted_nodes[a], sorted_nodes[b] = sorted_nodes[b], sorted_nodes[a]
                    node_index[sorted_nodes[a]], node_index[sorted_nodes[b]] = node_index[sorted_nodes[b]], node_index[sorted_nodes[a]]
                    start_degrees_index[v_degree] += 1
                    if not visited_neighbour:
                        start_degrees_index[u_degree] += 1
                else:
                    a = i + 1
                    b = int(node_index[v])
                    sorted_nodes[a], sorted_nodes[b] = sorted_nodes[b], sorted_nodes[a]
                    node_index[sorted_nodes[a]], node_index[sorted_nodes[b]] = node_index[sorted_nodes[b]], node_index[sorted_nodes[a]]
                    start_degrees_index[v_degree] += 2
                    start_degrees_index[v_degree - 1] += 1
                visited_neighbour = True

        #print(degrees)
        #print(sorted_nodes)
        #print("---")
    return c, nodes_core, ord

In [41]:
list(nx.degree(G))

[(0, 4), (1, 5), (2, 4), (3, 3), (4, 2), (5, 4), (6, 2), (7, 1), (8, 1)]

In [42]:
def cherchemin(d):
    m,md = d[0]
    for x,y in d:
        if y == 0:
            return x,y
        if y <= md:
            md = y
            m = x
    return m,md

In [43]:
c,core,ord1 = Core_decompositionfinal(G)
ord1

[3, 2, 1, 4, 6, 5, 7, 9, 8]

In [44]:
def densest_prefix(ordering,G):
    edges = list(G.edges)
    res = list(np.zeros(len(ordering),int))
    
    for (e1,e2) in edges:
        e = max(ordering[e1],ordering[e2])
        res[e-1] = res[e-1] + 1
        
    #print(res)
    for i in range(0,len(res)):
        if i != 0:
            res[i] = (res[i] + res[i-1])
            
    #print(res)
    for i in range(0,len(res)):
        if i != 0:
            res[i] = res[i]/(i+1)
    
    return res

In [46]:
densest_prefix(ord1,G)

[0,
 0.5,
 1.0,
 1.5,
 1.6,
 1.6666666666666667,
 1.5714285714285714,
 1.5,
 1.4444444444444444]

In [47]:
list(nx.core_number(G).values())

[3, 3, 3, 3, 2, 2, 1, 1, 1]

In [48]:
nx.k_core(G).edges()

EdgeView([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)])

In [53]:
def tme4_q1(filename):
    time_start = time.time()
    G,edges,nodes = load_graph(filename)
    print("loadgraph:",filename )
    
    add = len(G.edges)/len(G.nodes)
    print("average degree density :", add)
    
    ed = (2*len(G.edges))/(len(G.nodes)*(len(G.nodes)-1))
    print("the edge density :", ed)
    
    #core = list(nx.core_number(G).values())
    #print("core value :", max(core))
    time_start = time.time()
    c,core,ord1 = Core_decompositionfinal(G)
    print("core value :",c)
    
    densest_p = densest_prefix(ord1,G)
    max_densest_p = max(densest_p)
    print("maximum value of densest prefixe:", max_densest_p)

    k = densest_p.index(max_densest_p)+1
    print("the size of a densest core ordering prefixe:",k)
        
    time_end = time.time()
    print("Charge time:", time_end - time_start, "seconds")


In [54]:
tme4_q1('com-amazon.ungraph.txt')
#tme4_q1('test.txt')

loadgraph: com-amazon.ungraph.txt
average degree density : 2.7649277465709856
the edge density : 1.6513834036534368e-05
core value : 6
maximum value of densest prefixe: 3.9166666666666665
the size of a densest core ordering prefixe: 48
Charge time: 5.13730001449585 seconds
